In [ ]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from IPython.display import display_html
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import nltk
import re
nltk.download("punkt_tab") # new version
nltk.download('stopwords') 
nltk.download('wordnet')    
nltk.download('omw-1.4') 
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\alexc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\alexc\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\alexc\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\alexc\AppData\Roaming\nltk_data...
[nltk_data]   Package average

True

# Load the data

In [23]:
df = pd.read_csv('data/training_data.csv', sep="\t", header=None)

# Explore the data

In [24]:
print(df.shape)
df.head()

(34152, 2)


,0,1
0,0,donald trump sends out embarrassing new year‚s...
1,0,drunk bragging trump staffer started russian c...
2,0,sheriff david clarke becomes an internet joke ...
3,0,trump is so obsessed he even has obama‚s name ...
4,0,pope francis just called out donald trump duri...


In [25]:
df.columns = ['label', 'headline']

In [26]:
df.head()

,label,headline
0,0,donald trump sends out embarrassing new year‚s...
1,0,drunk bragging trump staffer started russian c...
2,0,sheriff david clarke becomes an internet joke ...
3,0,trump is so obsessed he even has obama‚s name ...
4,0,pope francis just called out donald trump duri...


### Look for empty rows

In [ ]:

df.isna().sum()

label       0
headline    0
dtype: int64

### Diference between features and target

In [28]:
X = df.drop('label', axis=1)
y = df['label']

# Split the data

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42, stratify=y)

# Process the data

Clean

In [34]:
def clean_text(text):
    
    # 1. Remove special characters
    text = re.sub(r'\W', ' ', text)
    # 2. Remove numbers
    text = re.sub(r'\d', ' ', text)
    # 3. Remove all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    # 4. Remove single characters from the start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    # 5. Substitute multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)
    # 6. Remove prefixed 'b'
    text = re.sub(r'^b\s+', '', text)
    # 7. Convert to lowercase
    text = text.lower()
    return text.strip()
# Aplicar ao DataFrame
X_train['text_clean'] = X_train['headline'].apply(clean_text)
X_test['text_clean'] = X_test['headline'].apply(clean_text)
print("\nAfter Cleaning:")
display(X_train)
display(X_test)


After Cleaning:


,headline,text_clean
6851,republicans punish georgia governor for refusi...,republicans punish georgia governor for refusi...
17313,father of soldier slain in niger defends presi...,father of soldier slain in niger defends presi...
22435,south dakota's governor vetoes loosening of co...,south dakota governor vetoes loosening of conc...
29488,turkey's erdogan says will take jerusalem reso...,turkey erdogan says will take jerusalem resolu...
6625,bill maher insults trump‚s supposed masculinit...,bill maher insults trump supposed masculinity ...
...,...,...
5267,trump sends hillary a pathetic threat just one...,trump sends hillary pathetic threat just one d...
13642,breaking: fbi investigating clinton crony virg...,breaking fbi investigating clinton crony virgi...
19391,new russian envoy describes 'warm' meeting wit...,new russian envoy describes warm meeting with ...
883,like father,like father


,headline,text_clean
10145,msnbc propagandist: the word ‚trump‚ is ‚moder...,msnbc propagandist the word trump is modern da...
26343,clinton says trump is most divisive candidate ...,clinton says trump is most divisive candidate ...
22173,ivanka trump becomes unpaid white house employee,ivanka trump becomes unpaid white house employee
365,trump supporters at ‚mother of all rallies‚ ma...,trump supporters at mother of all rallies mass...
13323,breaking: fresno police release graphic video ...,breaking fresno police release graphic video o...
...,...,...
19242,senate panel cancels interview with trump lawy...,senate panel cancels interview with trump lawy...
8508,bundy militant vows to die in gunfight with fe...,bundy militant vows to die in gunfight with fe...
21887,groups sue to obtain white house visitor logs,groups sue to obtain white house visitor logs
12582,breaking: 28 yr old palestinian muslim feras m...,breaking yr old palestinian muslim feras moham...


### Tokanization

In [ ]:
X_train['text_clean'] = X_train['text_clean'].apply(word_tokenize)
X_test['text_clean'] = X_test['text_clean'].apply(word_tokenize)

print("\nAfter Tokanizing:")
display(X_train)
display(X_test)


After Tokanizing:


,headline,text_clean
6851,republicans punish georgia governor for refusi...,"[republicans, punish, georgia, governor, for, ..."
17313,father of soldier slain in niger defends presi...,"[father, of, soldier, slain, in, niger, defend..."
22435,south dakota's governor vetoes loosening of co...,"[south, dakota, governor, vetoes, loosening, o..."
29488,turkey's erdogan says will take jerusalem reso...,"[turkey, erdogan, says, will, take, jerusalem,..."
6625,bill maher insults trump‚s supposed masculinit...,"[bill, maher, insults, trump, supposed, mascul..."
...,...,...
5267,trump sends hillary a pathetic threat just one...,"[trump, sends, hillary, pathetic, threat, just..."
13642,breaking: fbi investigating clinton crony virg...,"[breaking, fbi, investigating, clinton, crony,..."
19391,new russian envoy describes 'warm' meeting wit...,"[new, russian, envoy, describes, warm, meeting..."
883,like father,"[like, father]"


,headline,text_clean
10145,msnbc propagandist: the word ‚trump‚ is ‚moder...,"[msnbc, propagandist, the, word, trump, is, mo..."
26343,clinton says trump is most divisive candidate ...,"[clinton, says, trump, is, most, divisive, can..."
22173,ivanka trump becomes unpaid white house employee,"[ivanka, trump, becomes, unpaid, white, house,..."
365,trump supporters at ‚mother of all rallies‚ ma...,"[trump, supporters, at, mother, of, all, ralli..."
13323,breaking: fresno police release graphic video ...,"[breaking, fresno, police, release, graphic, v..."
...,...,...
19242,senate panel cancels interview with trump lawy...,"[senate, panel, cancels, interview, with, trum..."
8508,bundy militant vows to die in gunfight with fe...,"[bundy, militant, vows, to, die, in, gunfight,..."
21887,groups sue to obtain white house visitor logs,"[groups, sue, to, obtain, white, house, visito..."
12582,breaking: 28 yr old palestinian muslim feras m...,"[breaking, yr, old, palestinian, muslim, feras..."


### Stop Words

In [37]:
stop_words = set(stopwords.words('english'))

X_train['text_clean'] = X_train['text_clean'].apply(
    lambda tokens: [t for t in tokens if t not in stop_words]
)

X_test['text_clean'] = X_test['text_clean'].apply(
    lambda tokens: [t for t in tokens if t not in stop_words]
)

In [38]:
print("\nAfter Stop Words:")
display(X_train)
display(X_test)


After Stop Words:


,headline,text_clean
6851,republicans punish georgia governor for refusi...,"[republicans, punish, georgia, governor, refus..."
17313,father of soldier slain in niger defends presi...,"[father, soldier, slain, niger, defends, presi..."
22435,south dakota's governor vetoes loosening of co...,"[south, dakota, governor, vetoes, loosening, c..."
29488,turkey's erdogan says will take jerusalem reso...,"[turkey, erdogan, says, take, jerusalem, resol..."
6625,bill maher insults trump‚s supposed masculinit...,"[bill, maher, insults, trump, supposed, mascul..."
...,...,...
5267,trump sends hillary a pathetic threat just one...,"[trump, sends, hillary, pathetic, threat, one,..."
13642,breaking: fbi investigating clinton crony virg...,"[breaking, fbi, investigating, clinton, crony,..."
19391,new russian envoy describes 'warm' meeting wit...,"[new, russian, envoy, describes, warm, meeting..."
883,like father,"[like, father]"


,headline,text_clean
10145,msnbc propagandist: the word ‚trump‚ is ‚moder...,"[msnbc, propagandist, word, trump, modern, day..."
26343,clinton says trump is most divisive candidate ...,"[clinton, says, trump, divisive, candidate, li..."
22173,ivanka trump becomes unpaid white house employee,"[ivanka, trump, becomes, unpaid, white, house,..."
365,trump supporters at ‚mother of all rallies‚ ma...,"[trump, supporters, mother, rallies, massively..."
13323,breaking: fresno police release graphic video ...,"[breaking, fresno, police, release, graphic, v..."
...,...,...
19242,senate panel cancels interview with trump lawy...,"[senate, panel, cancels, interview, trump, law..."
8508,bundy militant vows to die in gunfight with fe...,"[bundy, militant, vows, die, gunfight, feds, g..."
21887,groups sue to obtain white house visitor logs,"[groups, sue, obtain, white, house, visitor, l..."
12582,breaking: 28 yr old palestinian muslim feras m...,"[breaking, yr, old, palestinian, muslim, feras..."


### POS tagging

In [39]:
X_train['text_clean'] = X_train['text_clean'].apply(nltk.pos_tag)
X_test['text_clean'] = X_test['text_clean'].apply(nltk.pos_tag)

In [40]:
print("\nAfter POS tagging:")
display(X_train)
display(X_test)


After POS tagging:


,headline,text_clean
6851,republicans punish georgia governor for refusi...,"[(republicans, NNS), (punish, JJ), (georgia, N..."
17313,father of soldier slain in niger defends presi...,"[(father, RB), (soldier, JJR), (slain, NN), (n..."
22435,south dakota's governor vetoes loosening of co...,"[(south, JJ), (dakota, JJ), (governor, NN), (v..."
29488,turkey's erdogan says will take jerusalem reso...,"[(turkey, NN), (erdogan, NN), (says, VBZ), (ta..."
6625,bill maher insults trump‚s supposed masculinit...,"[(bill, NN), (maher, CC), (insults, JJ), (trum..."
...,...,...
5267,trump sends hillary a pathetic threat just one...,"[(trump, NN), (sends, VBZ), (hillary, JJ), (pa..."
13642,breaking: fbi investigating clinton crony virg...,"[(breaking, VBG), (fbi, NN), (investigating, V..."
19391,new russian envoy describes 'warm' meeting wit...,"[(new, JJ), (russian, JJ), (envoy, NN), (descr..."
883,like father,"[(like, IN), (father, NN)]"


,headline,text_clean
10145,msnbc propagandist: the word ‚trump‚ is ‚moder...,"[(msnbc, JJ), (propagandist, NN), (word, NN), ..."
26343,clinton says trump is most divisive candidate ...,"[(clinton, NN), (says, VBZ), (trump, VB), (div..."
22173,ivanka trump becomes unpaid white house employee,"[(ivanka, JJ), (trump, NN), (becomes, NNS), (u..."
365,trump supporters at ‚mother of all rallies‚ ma...,"[(trump, NN), (supporters, NNS), (mother, CC),..."
13323,breaking: fresno police release graphic video ...,"[(breaking, VBG), (fresno, JJ), (police, NN), ..."
...,...,...
19242,senate panel cancels interview with trump lawy...,"[(senate, JJ), (panel, NN), (cancels, NNS), (i..."
8508,bundy militant vows to die in gunfight with fe...,"[(bundy, NN), (militant, NN), (vows, VBZ), (di..."
21887,groups sue to obtain white house visitor logs,"[(groups, NNS), (sue, VBP), (obtain, VB), (whi..."
12582,breaking: 28 yr old palestinian muslim feras m...,"[(breaking, VBG), (yr, PRP), (old, JJ), (pales..."


### Lemmatization

In [42]:
lemmatizer = WordNetLemmatizer()

X_train['text_clean'] = X_train['text_clean'].apply(
    lambda tokens: [lemmatizer.lemmatize(t[0]) for t in tokens]
)

X_test['text_clean'] = X_test['text_clean'].apply(
    lambda tokens: [lemmatizer.lemmatize(t[0]) for t in tokens]
)
print("\nAfter Lemmatization:")
display(X_train)
display(X_test)


After Lemmatization:


,headline,text_clean
6851,republicans punish georgia governor for refusi...,"[republican, punish, georgia, governor, refusi..."
17313,father of soldier slain in niger defends presi...,"[father, soldier, slain, niger, defends, presi..."
22435,south dakota's governor vetoes loosening of co...,"[south, dakota, governor, veto, loosening, con..."
29488,turkey's erdogan says will take jerusalem reso...,"[turkey, erdogan, say, take, jerusalem, resolu..."
6625,bill maher insults trump‚s supposed masculinit...,"[bill, maher, insult, trump, supposed, masculi..."
...,...,...
5267,trump sends hillary a pathetic threat just one...,"[trump, sends, hillary, pathetic, threat, one,..."
13642,breaking: fbi investigating clinton crony virg...,"[breaking, fbi, investigating, clinton, crony,..."
19391,new russian envoy describes 'warm' meeting wit...,"[new, russian, envoy, describes, warm, meeting..."
883,like father,"[like, father]"


,headline,text_clean
10145,msnbc propagandist: the word ‚trump‚ is ‚moder...,"[msnbc, propagandist, word, trump, modern, day..."
26343,clinton says trump is most divisive candidate ...,"[clinton, say, trump, divisive, candidate, lif..."
22173,ivanka trump becomes unpaid white house employee,"[ivanka, trump, becomes, unpaid, white, house,..."
365,trump supporters at ‚mother of all rallies‚ ma...,"[trump, supporter, mother, rally, massively, o..."
13323,breaking: fresno police release graphic video ...,"[breaking, fresno, police, release, graphic, v..."
...,...,...
19242,senate panel cancels interview with trump lawy...,"[senate, panel, cancel, interview, trump, lawy..."
8508,bundy militant vows to die in gunfight with fe...,"[bundy, militant, vow, die, gunfight, fed, guy..."
21887,groups sue to obtain white house visitor logs,"[group, sue, obtain, white, house, visitor, log]"
12582,breaking: 28 yr old palestinian muslim feras m...,"[breaking, yr, old, palestinian, muslim, feras..."


# Vectorization

## TF-IDF

In [ ]:
# # Generate classification report as a dict
# report_dict = classification_report(y_true, y_pred, output_dict=True)

# # Convert to DataFrame
# report_df = pd.DataFrame(report_dict).transpose()

# # Save to CSV
# report_df.to_csv("classification_report.csv", index=True)